# 다양한 단어의 표현 방법

## 1. 단어의 표현 방법
- 국소 표현 (Local Representation) 
    - 해당 단어 그 자체로, 특정값을 맵핑하여 단어를 표현하는 방법
    - 이산 표현(Discrete Representation)
- 분산 표현 (Distributed Representation)
    - 해당 단어 주변을 참고하여 단어를 표현하는 방법
    - 뉘앙스 표현 가능
    - 연속 표현(Continuous Representation)
    

- 예시)
    - puppy, cute, lovely -> 각 단어를 맵핑하면 국소 표현 방법
    - puppy 주변에 cute, lovely 가 자주 등장하므로, 두 단어는 연관성이 있다. ->

## 2. 단어 표현의 카테고리 화

![카테고리](https://wikidocs.net/images/page/31767/wordrepresentation.PNG)

# Bag of Words (BoW)
- 단어들의 순서를 고려하지 않고, 단어들의 출현 빈도에 집중하는 텍스트 데이터의 수치화 표현 방법
- 문서의 성격을 판단하는 작업에 쓰인다. (빈도가 높은 단어들을 중점으로 파악)


- Bag of Words는 국소 표현
    - 단어의 빈도수를 카운트하여 단어를 수치화하는 단어 표현 방법
- 절차
    1. 각 단어에 고유한 정수 인덱스를 부여
    2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터 생성

In [18]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(doc):
    doc = doc.replace('.', '')
    tokenized_doc = okt.morphs(doc)
    
    word_to_index = {}
    bow = []
    
    for word in tokenized_doc:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # BoW 에 기본값 1을 넣는다
            bow.insert(len(word_to_index)-1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            bow[index] = bow[index]+1
    return word_to_index, bow

[0.004s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016b7c8000-0x000000016b7d4000).
[0.004s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [19]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print("Vocab : ", vocab)
print("Bag of Words vector : ", bow)

Vocab :  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
Bag of Words vector :  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [20]:
doc2 = "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다."
vocab, bow = build_bag_of_words(doc2)
print("Vocab : ", vocab)
print("Bag of Words vector : ", bow)

Vocab :  {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
Bag of Words vector :  [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [21]:
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print("Vocab : ", vocab)
print("Bag of Words vector : ", bow)

Vocab :  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
Bag of Words vector :  [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


## 3. CounteVectorizer 클래스로 BoW 만들기

- 사이킷 런에서 단어의 빈도를 count 하여 Vector 로 만들어줌
    - CountVectorizer 클래스
- 길이가 2 이상이 문자에 대해서만 토큰으로 인식해준다
    - 영어에서는 짧은 문자를 제거하는 것도 전처리 작업으로 고려됨
- 주의점
    - 띄어쓰기를 기준으로 단어를 자르기 때문에 수준이 낮음
    - 한글에서 쓸 경우 원하는 의도와 다르게 될 가능성이 있음

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print("Bag of Words Vector : ", vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지 출력
print("Vocab : ", sorted(vector.vocabulary_.items(), key= lambda item : item[1]))

Bag of Words Vector :  [[1 1 2 1 2 1]]
Vocab :  [('because', 0), ('know', 1), ('love', 2), ('want', 3), ('you', 4), ('your', 5)]


In [26]:
# 한글도 정상적으로 먹히지만, 조사나 데이터 전처리 후 하는게 좋겠군
corpus = ['정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print("Bag of Words Vector : ", vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지 출력
print("Vocab : ", vector.vocabulary_)

Bag of Words Vector :  [[1 1 1 1 1 1 1]]
Vocab :  {'정부가': 6, '발표하는': 4, '물가상승률과': 2, '소비자가': 5, '느끼는': 0, '물가상승률은': 3, '다르다': 1}


## 4. 불용어를 제거한 BoW 만들기
- BoW는 해당 문서에 각 단어가 얼마나 등장하는지 파악하기 위함
    - 불용어를 제거하는 것은, 자연어 처리의 정확도를 높이기 위해 선택 가능


- 영어의 BoW 를 만들기 위해 불용어를 제외하고 만드는 방법!!!!

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# 1. 사용자가 직접 정의한 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=['the','a','an','is','not'])
print("Bag of Words vector : ", vect.fit_transform(text).toarray())
print("Vocab :", vect.vocabulary_)



Bag of Words vector :  [[1 1 1 1 1]]
Vocab : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [28]:
# 2. CountVectorizer 에서 제공하는 자체 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words='english')
print("Bag of Words vector : ", vect.fit_transform(text).toarray())
print("Vocab :", vect.vocabulary_)



Bag of Words vector :  [[1 1 1]]
Vocab : {'family': 0, 'important': 1, 'thing': 2}


In [29]:
# 3. NLTK 에서 지원하는 불용어 사용
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words('english')
vect = CountVectorizer(stop_words=stop_words)
print("Bag of Words vector : ", vect.fit_transform(text).toarray())
print("Vocab :", vect.vocabulary_)



Bag of Words vector :  [[1 1 1 1]]
Vocab : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}
